In [ ]:
# folders
data_path = 'xxx' # todo
results_path = 'xxx' # todo

# Try different distributions - NbrData experiments

In [ ]:
import matplotlib.pyplot as plt
from utils import set_seeds
import numpy as np
import pandas as pd

train_val = 1000*np.ones((10,5))
cnt = 0

CLASS_NAMES = []
CLASS_NAMES.append(range(0,10))
CLASS_NAMES.append(range(0,5))

# initialize
df_datapoints = pd.DataFrame(columns = ['mean_organ', 
                                        'std_organ', 
                                        'mean_modality',
                                        'std_modality',
                                        'mean_datapoints',
                                        'median_datapoints',
                                        'total_datapoints'])
    
for mean_organ in [0]:
    for std_organ in [3,5,9,17]:
        
        # distribution for organ
        set_seeds(seed=42)
        s_organ = np.random.normal(mean_organ, std_organ, 500000)
        
        for mean_modality in [0,2]:
            for std_modality in [1,3,5]:
                
                # distribution for modality
                set_seeds(seed=42)
                s_modality = np.random.normal(mean_modality, std_modality, 100000)
                
                # initialize
                hist_organ_mat = np.empty([len(CLASS_NAMES[0]),len(CLASS_NAMES[1])])
                hist_modality_mat = np.empty([len(CLASS_NAMES[0]),len(CLASS_NAMES[1])])
                
                # histogram
                hist_organ = np.histogram(s_organ, bins=range(0,11), density=True)
                hist_modality = np.histogram(s_modality, bins=range(0,6), density=True)
                
                # normalize
                hist_organ = hist_organ[0]/hist_organ[0].max()
                hist_modality = hist_modality[0]/hist_modality[0].max()
                
                # iterate 
                for i in range(len(CLASS_NAMES[1])):
                    hist_organ_mat[:,i] = hist_organ
                for i in range(len(CLASS_NAMES[0])):
                    hist_modality_mat[i,:] = hist_modality

                # number of datapoints
                datapoints = train_val*hist_organ_mat*hist_modality_mat
                mean_datapoints = datapoints.mean()
                median_datapoints = np.median(datapoints)
                total_datapoints = datapoints.sum()
                print(median_datapoints,mean_datapoints,total_datapoints)
                df_datapoints = pd.concat([df_datapoints, pd.DataFrame.from_records([{'mean_organ':mean_organ, 
                                                     'std_organ': std_organ,
                                                     'mean_modality':mean_modality,
                                                     'std_modality':std_modality,
                                                     'mean_datapoints':mean_datapoints,
                                                     'median_datapoints':median_datapoints,
                                                     'total_datapoints':total_datapoints}])], ignore_index=True)
            
                # plot
                fig = plt.figure(figsize=(7, 3), dpi=80)
                
                # plot
                plt.subplot(1,3,1)
                plt.imshow(hist_organ_mat, vmin=0, vmax=1)
                plt.title('Mean: '+str(mean_organ)+ ' Std: '+str(std_organ))
                plt.colorbar()
                plt.subplot(1,3,2)
                plt.imshow(hist_modality_mat, vmin=0, vmax=1)
                plt.title('Mean: '+str(mean_modality)+ ' Std: '+str(std_modality))
                plt.colorbar()
                plt.subplot(1,3,3)
                img = plt.imshow(datapoints, vmin=0, vmax=1000)
                plt.title('Number of samples')
                plt.colorbar()
                plt.show()
                
                cnt = cnt + 1
print(cnt)     
df_datapoints = df_datapoints.sort_values('median_datapoints',ignore_index=True)

In [ ]:
import pandas as pd
import numpy as np
from utils import set_seeds
import matplotlib.pyplot as plt

# load data
data_dir = data_path + '/Data/PolyMNIST/' 
curr_phase = 'train'
max_id = 1000
train_ratio = 0.75
seed = 42
mean_organ = 0
std_organ = 3
mean_modality = 0
std_modality = 5

CLASS_NAMES = []
CLASS_NAMES.append(range(0,10))
CLASS_NAMES.append(range(0,5))

num_classes = np.zeros(len(CLASS_NAMES))
for i in range(len(CLASS_NAMES)):
    num_classes[i] = len(CLASS_NAMES[i])
num_classes = num_classes.astype(int)
num_classes_total = int(sum(num_classes))

annot_dir = data_dir + curr_phase + '_' + str(max_id) + '.csv'
df = pd.read_csv(annot_dir,sep=';') 
imgs_dir = data_dir + curr_phase + '_' + str(max_id) + '.npy'
imgs = np.load(imgs_dir)

# get img_idx and labels
img_idx = np.array(df['Path'])
label_vec = np.transpose(np.array([df['Organ'],df['Modality']]))
label_vec = label_vec.astype(int)
            
# get the current distribution
set_seeds(seed=seed)
s_organ = np.random.normal(mean_organ, std_organ, 500000)
set_seeds(seed=seed)
s_modality = np.random.normal(mean_modality, std_modality, 100000)
            
# initialize
hist_organ_mat = np.empty([num_classes[0],num_classes[1]])
hist_modality_mat = np.empty([num_classes[0],num_classes[1]])
                
# histogram
hist_organ = np.histogram(s_organ, bins=range(0,num_classes[0]+1), density=True)
hist_modality = np.histogram(s_modality, bins=range(0,num_classes[1]+1), density=True)
                
# normalize
hist_organ = hist_organ[0]/hist_organ[0].max()
hist_modality = hist_modality[0]/hist_modality[0].max()
                
# iterate 
for i in range(len(CLASS_NAMES[1])):
    hist_organ_mat[:,i] = hist_organ
for i in range(len(CLASS_NAMES[0])):
    hist_modality_mat[i,:] = hist_modality

# number of datapoints
train_mat = max_id*train_ratio*np.ones((num_classes[0],num_classes[1]))
val_mat =  max_id*(1-train_ratio)*np.ones((num_classes[0],num_classes[1]))
datapoints_train = np.round(train_mat*hist_organ_mat*hist_modality_mat)
datapoints_val = np.round(val_mat*hist_organ_mat*hist_modality_mat)

print('Total number of images for train: '+str(datapoints_train.sum()))
print('Total number of images for val: '+str(datapoints_val.sum()))
print(datapoints_train)
print(datapoints_train.sum(axis=1))
print(datapoints_val)
print(datapoints_val.sum(axis=1))
                                             
# get only the relevant images
# train
check_vec = df['ImgID'] <  datapoints_train[df['Organ'],df['Modality']] 
train_imgs = imgs[:,:,:,check_vec]
train_idx = img_idx[check_vec]
train_label = label_vec[check_vec]
train_dir = data_dir + curr_phase + '/'
            
# val
check_vec = (df['ImgID'] < (max_id*train_ratio + datapoints_val[df['Organ'],df['Modality']])) & (df['ImgID'] >= (max_id*train_ratio))
val_imgs = imgs[:,:,:,check_vec]
val_idx = img_idx[check_vec]
val_label = label_vec[check_vec]
val_dir = data_dir + curr_phase + '/'

# Number of datapoints experiments

## Configs for nbrData experiments

In [ ]:
from config_experiments_mnist import config_experiments_nbrData
import json

# hyperparameter configs
config_experiments_nbrData(results_path + '/MNIST/results/results_NbrData/',True) 
with open(results_path + '/MNIST/results/results_NbrData/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)    

## run from terminal - nbrData experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='NbrData' --gpu_id=0

# NbrData Secondary experiments

## Configs for secondary experiment

In [ ]:
from config_experiments_mnist import config_experiments_nbrData_secondary
import json
# reduced experiment configs        
config_experiments_nbrData_secondary(results_path + '/MNIST/results/results_NbrData_Secondary/',True)
with open(results_path + '/MNIST/results/results_NbrData_Secondary/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)          

## run from terminal - secondary experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='NbrData_Secondary' --gpu_id=0

# NbrData Secondary experiments - upsampled

## Configs for secondary upsampled experiment

In [ ]:
from config_experiments_mnist import config_experiments_nbrData_secondary_upsampled
import json
# reduced experiment configs        
config_experiments_nbrData_secondary_upsampled(results_path + '/MNIST/results/results_NbrData_Secondary_Upsampled/',True) 
with open(results_path + '/MNIST/results/results_NbrData_Secondary_Upsampled/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)    

## run from terminal - secondary upsampled experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='NbrData_Secondary_Upsampled' --gpu_id=0

# NbrData Reduced experiments

## Configs for reduced experiments

In [ ]:
from config_experiments_mnist import config_experiments_nbrData_reduced
import json        
        
# reduced experiment configs        
config_experiments_nbrData_reduced(results_path + '/MNIST/results/results_NbrData_Reduced/',True) 
with open(results_path + '/MNIST/results/results_NbrData_Reduced/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)              

## run from terminal - reduced experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='NbrData_Reduced' --gpu_id=0

# NbrData Secondary Reduced experiments

## Configs for reduced secondary experiments

In [ ]:
from config_experiments_mnist import config_experiments_nbrData_reduced_secondary
import json

# hyperparameter configs
config_experiments_nbrData_reduced_secondary(results_path + '/MNIST/results/results_NbrData_ReducedSecondary/',True) 
with open(results_path + '/MNIST/results/results_NbrData_ReducedSecondary/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)  

## run from terminal - reduced secondary experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='NbrData_ReducedSecondary' --gpu_id=0

# NbrData Secondary Reduced experiments - upsampled 

## Configs for reduced secondary upsampled experiments

In [ ]:
from config_experiments_mnist import config_experiments_nbrData_reduced_secondary_upsampled
import json

# hyperparameter configs
config_experiments_nbrData_reduced_secondary_upsampled(results_path + '/MNIST/results/results_NbrData_ReducedSecondary_upsampled/',True) 
with open(results_path + '/MNIST/results/results_NbrData_ReducedSecondary_upsampled/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)  

## run from terminal - reduced secondary upsampled experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='NbrData_ReducedSecondary_upsampled' --gpu_id=0

## NbrData Secondary Reduced experiments - test on other modalities

In [ ]:
from config_experiments_mnist import config_experiments_nbrData_reduced_secondary_test
import json

# hyperparameter configs
config_experiments_nbrData_reduced_secondary_test(results_path + '/MNIST/results/results_NbrData_ReducedSecondary_test/',True) 
with open(results_path + '/MNIST/results/results_NbrData_ReducedSecondary_test/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)  

## run from terminal - reduced secondary test experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='NbrData_ReducedSecondary_test' --gpu_id=0

# Sampled datapoints experiments

## Configs for sampled experiments

In [ ]:
from config_experiments_mnist import config_experiments_sampled
import json

# hyperparameter configs
config_experiments_sampled(results_path + '/MNIST/results/results_Sampled/',True) 
with open(results_path + '/MNIST/results/results_Sampled/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)    

## run from terminal - sampled experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='Sampled' --gpu_id=0

# Sampled Secondary experiments

## Configs for secondary experiment

In [ ]:
from config_experiments_mnist import config_experiments_sampled_secondary
import json
# reduced experiment configs        
config_experiments_sampled_secondary(results_path + '/MNIST/results/results_Sampled_Secondary/',True) 
with open(results_path + '/MNIST/results/results_Sampled_Secondary/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)          

## run from terminal - secondary experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='Sampled_Secondary' --gpu_id=0

# Sampled Secondary experiments - upsampled

## Configs for secondary upsampled experiment

In [ ]:
from config_experiments_mnist import config_experiments_sampled_secondary_upsampled
import json
# reduced experiment configs        
config_experiments_sampled_secondary_upsampled(results_path + '/MNIST/results/results_Sampled_Secondary_Upsampled/',True) 
with open(results_path + '/MNIST/results/results_Sampled_Secondary_Upsampled/' + 'configs/' + str(0) + '.json') as config_file: 
        config = json.load(config_file)
        print(config)    

## run from terminal - secondary upsampled experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='Sampled_Secondary_Upsampled' --gpu_id=0

# Sampled Reduced experiments

## Configs for reduced experiments

In [ ]:
from config_experiments_mnist import config_experiments_sampled_reduced
import json        
        
# reduced experiment configs        
config_experiments_sampled_reduced(results_path + '/MNIST/results/results_Sampled_Reduced/',True)
with open(results_path + '/MNIST/results/results_Sampled_Reduced/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)              

## run from terminal - reduced experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='Sampled_Reduced' --gpu_id=0

# Sampled Secondary Reduced experiments

## Configs for reduced secondary experiments

In [ ]:
from config_experiments_mnist import config_experiments_sampled_reduced_secondary
import json

# hyperparameter configs
config_experiments_sampled_reduced_secondary(results_path + '/MNIST/results/results_Sampled_ReducedSecondary/',True)
with open(results_path + '/MNIST/results/results_Sampled_ReducedSecondary/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)  

## run from terminal - reduced secondary experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='Sampled_ReducedSecondary' --gpu_id=0

# Sampled Secondary Reduced experiments - upsampled

## Configs for reduced secondary upsampled experiments

In [ ]:
from config_experiments_mnist import config_experiments_sampled_reduced_secondary_upsampled
import json

# hyperparameter configs
config_experiments_sampled_reduced_secondary_upsampled(results_path + 'MNIST/results/results_Sampled_ReducedSecondary_upsampled/',True)
with open(results_path + '/MNIST/results/results_Sampled_ReducedSecondary_upsampled/' + 'configs/' + str(0) + '.json') as config_file:
        config = json.load(config_file)
        print(config)  

## run from terminal - reduced secondary upsampled experiments

In [ ]:
!python main_mnist.py --experiment_id=0 --task_to_run='Sampled_ReducedSecondary_upsampled' --gpu_id=0